In [1]:
using Gurobi, JuMP, CSV, DataFrames, Random, Distributions

In [2]:
utility_B = CSV.read("U_B.csv", DataFrame, pool=true);
utility_R = CSV.read("U_R.csv", DataFrame, pool=true);

In [3]:
utility_B

Row,champion,bot,jng,mid,sup,top
,String15,Float64,Float64,Float64,Float64,Float64
1,Aatrox,0.0,0.0,0.0,0.0,1.0
2,Ahri,0.0,0.0,3.16228,0.0,0.0
3,Akali,0.0,0.0,1.0,0.0,1.41421
4,Alistar,0.0,0.0,0.0,1.0,0.0
5,Amumu,0.0,0.0,0.0,2.0,0.0
6,Aphelios,3.0,0.0,0.0,0.0,0.0
7,Azir,0.0,0.0,1.41421,0.0,0.0
8,Braum,0.0,0.0,0.0,1.0,0.0
9,Corki,0.0,0.0,1.0,0.0,0.0


In [3]:
Ban_B = CSV.read("B_B.csv", DataFrame, pool=true);
Ban_R = CSV.read("B_R.csv", DataFrame, pool=true);

In [4]:
using Gurobi, JuMP, CSV, DataFrames, Random, Distributions

function robust_draft(U_B, U_R, B_B, B_R, sigma_RS, sigma_RB, gamma)
    n = size(U_B, 1)
    P = 5

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

   eps =0.0001
    @variable(model, BS[1:n, 1:P], Bin) # Blue Select
    @variable(model, RS[1:n, 1:P]) # Red Select
    @variable(model, BB[1:n], Bin) # Blue Ban 
    @variable(model, RB[1:n]) # Red Ban
    @variable(model, U_diff>=0)
    @variable(model, pS[1:n, 1:P]>=0)
    #@variable(model, qS[1:n, 1:P]>=0)
    @variable(model, pB[1:n]>=0)
    #@variable(model, qB[1:n]>=0)
    #constraints: 
    
    # Relaxation of Red team variables for robust
    @constraint(model, [1:n], RB.>= 0)
    @constraint(model, [1:n], RB.<= 1)
    @constraint(model, [1:n, 1:P], RS.<= 1)
    @constraint(model, [1:n, 1:P], RS.>= 0)

    # Banning exatctly 5 champions for both teams
    @constraint(model, sum(BB) <= 5)
    @constraint(model, sum(RB) <= 5)


   # Selection constraint per position for both teams
    @constraint(model, [p= 1:P], sum(RS[i,p] for i = 1:n) == 1 )
    @constraint(model, [p= 1:P], sum(BS[i,p] for i = 1:n) == 1 )

    # A champion can be selected or banned at most once per both teams: 
    @constraint(model, [i= 1:n], sum(BS[i,p] for p = 1:P) +sum(RS[i,p] for p = 1:P) + BB[i] + RB[i] <= 1)



    @constraint(model, [p=1:P], sum(RS[i, p]*U_R[i,p] for i=1:n) >= eps ) 
    #@constraint(model, [i= 1:n], sum(B_R[i]*RB[i] for i=1:n)>= eps)
    
    @constraint(model, [p=1:P], sum(BS[i, p]*U_B[i,p] for i=1:n) >= eps  ) 
    @constraint(model, [i= 1:n], sum(B_B[i]*BB[i] for i=1:n)>= eps)
    # Robust constraints: 
    @constraint(model, U_diff <= 
         sum(sum(BS[i, p]*U_B[i,p] for i=1:n) for p= 1:P) + sum(B_R[i]*BB[i] for i=1:n)
       + (sum(sum(RS[i, p]*U_R[i,p] for i=1:n) for p= 1:P) + sum(B_B[i]*RB[i] for i=1:n)) 
       - (sqrt(n)*gamma*sum((pS).*sigma_RS) + sqrt(n)*gamma*sum((pB).*sigma_RB))
       )

    @constraint(model, sum(sum(BS[i, p]*U_B[i,p] for i=1:n) for p= 1:P) + sum(B_R[i]*BB[i] for i=1:n) >= (sum(sum(RS[i, p]*U_R[i,p] for i=1:n) for p= 1:P) + sum(B_B[i]*RB[i] for i=1:n)) 
    - (sqrt(n)*gamma*sum((pS).*sigma_RS) + sqrt(n)*gamma*sum((pB).*sigma_RB)))
    
    @constraint(model, [i = 1:n, p = 1:P], pS[i,p] == RS[i,p])
    @constraint(model, [i = 1:n], pB[i] == RB[i])


    @objective(model, Max, U_diff)
    


    optimize!(model)

    return model, value.(BS), value.(BB), value.(RS), value.(RB), value.(sum(sum(BS[i, p]*U_B[i,p] for i=1:n) for p= 1:P) + sum(B_R[i]*BB[i] for i=1:n)), value.((sum(sum(RS[i, p]*U_R[i,p] for i=1:n) for p= 1:P) + sum(B_B[i]*RB[i] for i=1:n))), objective_value(model)
    end;


In [5]:



U_B = Matrix(utility_B[:,2:end]);
U_R = Matrix(utility_R[:,2:end]);

B_B = Matrix(Ban_B[:,2:end]);
B_R = Matrix(Ban_R[:,2:end]);


Random.seed!(42);
d = Normal(0,0.5);
sigma_s = abs.(rand(d, 62*5));
sigma_RS = reshape(sigma_s, 62, 5);
d = Normal(0,0.5);
sigma_RB = abs.(rand(d, 62));

for i =1:62
    for p = 1:5
        if U_R[i,p] ==0
            sigma_RS[i,p] = 0
        end;
    end;
end;

for i =1:62
        if B_R[i] ==0
            sigma_RB[i] = 0
    end;
end;
        



In [6]:
m, bs, bb, rs, rb, obj3 = robust_draft(U_B, U_R, B_B, B_R, sigma_RS, sigma_RB, 3)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-15


(A JuMP Model
Maximization problem with:
Variables: 1117
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 382 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 100017 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 100009 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 373 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 372 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: BB, BS, RB, RS, U_diff, pB, pS, [0.0 0.0 … 0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; -0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0  …  -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 37.760354233608425, 

In [7]:
m, bs, bb, rs, rb, obj2 = robust_draft(U_B, U_R, B_B, B_R, sigma_RS, sigma_RB, 2)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-15


(A JuMP Model
Maximization problem with:
Variables: 1117
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 382 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 100017 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 100009 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 373 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 372 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: BB, BS, RB, RS, U_diff, pB, pS, [0.0 0.0 … 0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0  …  -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 37.527354683433465, 2

In [8]:
m, bs, bb, rs, rb, obj1 = robust_draft(U_B, U_R, B_B, B_R, sigma_RS, sigma_RB, 1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-15


(A JuMP Model
Maximization problem with:
Variables: 1117
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 382 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 100017 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 100009 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 373 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 372 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: BB, BS, RB, RS, U_diff, pB, pS, [0.0 0.0 … 0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0  …  -0.0, 0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], 36.277554644181905, 34.

In [9]:
m, bs, bb, rs, rb, obj0 = robust_draft(U_B, U_R, B_B, B_R, sigma_RS, sigma_RB, 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-15


(A JuMP Model
Maximization problem with:
Variables: 1117
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 382 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 100017 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 100009 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 373 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 372 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: BB, BS, RB, RS, U_diff, pB, pS, [0.0 0.0 … 0.0 -0.0; 0.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0  …  -0.0, 1.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 37.45003313844286, 36.6

In [13]:
function get_selection(data)
    selection = []
    for j= 1:size(data,2)
        append!(selection, findall(x->x==1, (data[:,j])))
    end;
    return selection;
end;


In [14]:
blue_s = get_selection(bs)

5-element Vector{Any}:
 61
 49
 52
 34
 17

In [15]:
red_s = get_selection(rs)

5-element Vector{Any}:
  6
 54
  2
 29
 36

In [16]:
blue_b = findall(x -> x == 1, bb)

5-element Vector{Int64}:
  7
 16
 43
 55
 57

In [17]:
red_b = findall(x -> x == 1, rb)

5-element Vector{Int64}:
 14
 23
 28
 46
 59

In [18]:
BS = utility_B[blue_s, :][:,1];
RS = utility_R[red_s, :][:,1];
BB = utility_B[blue_b, :][:,1];
RB = utility_R[red_b, :][:,1];

In [19]:
utility_R[red_s, :]

Row,champion,bot,jng,mid,sup,top
,String15,Float64,Float64,Float64,Float64,Float64
1,Aphelios,5.56776,0.0,0.0,0.0,0.0
2,Viego,0.0,4.58258,0.0,0.0,0.0
3,Ahri,0.0,0.0,3.87298,0.0,0.0
4,Leona,0.0,0.0,0.0,3.31662,0.0
5,Ornn,0.0,0.0,0.0,0.0,3.87298


In [20]:
utility_B[blue_s, :]

Row,champion,bot,jng,mid,sup,top
,String15,Float64,Float64,Float64,Float64,Float64
1,Zeri,3.4641,0.0,0.0,0.0,0.0
2,Trundle,0.0,2.82843,0.0,0.0,0.0
3,Vex,0.0,0.0,2.82843,0.0,0.0
4,Nautilus,0.0,0.0,0.0,4.12311,0.0
5,Gwen,0.0,0.0,0.0,0.0,3.31662


In [21]:
for bs in BS
    println("Blue team picked: ", bs)
end;
println("-----------------")
for rs in RS
    println("Red team picked: ", rs)
end;


Blue team picked: Zeri
Blue team picked: Trundle
Blue team picked: Vex
Blue team picked: Nautilus
Blue team picked: Gwen
-----------------
Red team picked: Aphelios
Red team picked: Viego
Red team picked: Ahri
Red team picked: Leona
Red team picked: Ornn


In [22]:
for bb in BB
    println("Blue team banned: ", bb)
end;
println("-----------------")
for rb in RB
    println("Red team banned: ", rb)
end;

Blue team banned: Azir
Blue team banned: Graves
Blue team banned: Seraphine
Blue team banned: Viktor
Blue team banned: Wukong
-----------------
Red team banned: Gnar
Red team banned: Jinx
Red team banned: Lee Sin
Red team banned: Tahm Kench
Red team banned: Xin Zhao


In [29]:
println("Objective value for Gamma = 3: ", obj3)
println("Objective value for Gamma = 2: ", obj2)
println("Objective value for Gamma = 1: ", obj1)
println("Objective value for Gamma = 0: ", obj0)

Objective value for Gamma = 3: 37.760354233608425
Objective value for Gamma = 2: 37.527354683433465
Objective value for Gamma = 1: 36.277554644181905
Objective value for Gamma = 0: 35.775200502772755


In [30]:
findall(x-> x==1 , rs[:,5])

1-element Vector{Int64}:
 36

In [1]:
function robust_draft_sigma(U_B, U_R, B_B, B_R, sigma_RS, sigma_RB, gamma)
    n = size(U_B, 1)
    P = 5

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

   eps =0.0001
    @variable(model, BS[1:n, 1:P], Bin) # Blue Select
    @variable(model, RS[1:n, 1:P], Bin) # Red Select
    @variable(model, BB[1:n], Bin) # Blue Ban 
    @variable(model, RB[1:n], Bin) # Red Ban
    @variable(model, U_diff>=0)
    @variable(model, pS[1:n, 1:P]>=0)
    @variable(model, qS[1:n, 1:P]>=0)
    @variable(model, pB[1:n]>=0)
    @variable(model, qB[1:n]>=0)
    #constraints: 
    # Banning exatctly 5 champions for both teams
    @constraint(model, sum(BB) <= 5)
    @constraint(model, sum(RB) <= 5)
    # Selection constraint per champion for both teams
    
   # Selection constraint per position for both teams
   
    @constraint(model, [p= 1:P], sum(RS[i,p] for i = 1:n) == 1 )
    @constraint(model, [p= 1:P], sum(BS[i,p] for i = 1:n) == 1 )

    # A champion can be selected or banned at most once per both teams: 
    @constraint(model, [i= 1:n], sum(BS[i,p] for p = 1:P) +sum(RS[i,p] for p = 1:P) + BB[i] + RB[i] <= 1)



    @constraint(model, [p=1:P], sum(RS[i, p]*U_R[i,p] for i=1:n) >= eps ) 
    #@constraint(model, [i= 1:n], sum(B_R[i]*RB[i] for i=1:n)>= eps)
    
    @constraint(model, [p=1:P], sum(BS[i, p]*U_B[i,p] for i=1:n) >= eps  ) 
    @constraint(model, [i= 1:n], sum(B_B[i]*BB[i] for i=1:n)>= eps)
    # Robust constraints: 
    @constraint(model, U_diff <= 
         sum(sum(BS[i, p]*U_B[i,p] for i=1:n) for p= 1:P) + sum(B_B[i]*BB[i] for i=1:n)
       + (sum(sum(RS[i, p]*U_R[i,p] for i=1:n) for p= 1:P) + sum(B_R[i]*RB[i] for i=1:n) 
       + sqrt(n)*gamma*sum((qS.-pS).*sigma_RS) + sqrt(n)*gamma*sum((qB.-pB).*sigma_RB)
       ) )
    
    @constraint(model, [i = 1:n, p = 1:P], -pS[i,p]+ qS[i,p] == RS[i,p])
    @constraint(model, [i = 1:n], -pB[i]+ qB[i] == RB[i])


    @objective(model, Max, U_diff)
    


    optimize!(model)

    return model, value.(BS), value.(BB), value.(RS), value.(RB), value.(sum(sum(BS[i, p]*U_B[i,p] for i=1:n) for p= 1:P) + sum(B_B[i]*BB[i] for i=1:n)), objective_value(model)
    end;


LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[1]:9

In [57]:
utility_R

Row,champion,bot,jng,mid,sup,top
,String15,Float64,Float64,Float64,Float64,Float64
1,Aatrox,0.0,0.0,0.0,0.0,1.73205
2,Ahri,0.0,0.0,3.87298,0.0,0.0
3,Akali,0.0,0.0,2.0,0.0,0.0
4,Alistar,0.0,0.0,0.0,2.44949,0.0
5,Amumu,0.0,0.0,0.0,1.41421,0.0
6,Aphelios,5.56776,0.0,0.0,0.0,0.0
7,Azir,0.0,0.0,3.31662,0.0,0.0
8,Braum,0.0,0.0,0.0,2.44949,0.0
9,Corki,0.0,0.0,3.31662,0.0,0.0
